In [6]:
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import os
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
llm = AzureChatOpenAI(
    deployment_name= "gpt-4o",
    api_key=os.getenv('AZURE_OPENAI_API_KEY'),
    api_version="2023-09-15-preview",
    azure_endpoint="https://acsstscdamoai02.openai.azure.com/"
)

In [8]:
from langchain_core.messages import HumanMessage
llm.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content='Hi Bob! How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 11, 'total_tokens': 21, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_04751d0b65', 'finish_reason': 'stop', 'logprobs': None}, id='run-21b36b52-0824-40d2-b0d6-e0036ce1156e-0')

In [12]:
import re

def convert_to_polars_expr(input_str):
    # Step 1: Replace uppercase column names with `pl.col(...)`
    def replace_col(match):
        return f"pl.col('{match.group(0)}')"

    # Match uppercase variable names precisely
    replaced_str = re.sub(r'([A-Z]{1,2})', replace_col, input_str)
    print(f"After Column Replacement: {replaced_str}")

    # Step 2: Add multiplication where necessary
    replaced_str = re.sub(r'(\d)(\()', r'\1*(', replaced_str)  # Between number and '('
    print(f"After Adding * Before (: {replaced_str}")
    
    replaced_str = re.sub(r'\)(pl\.col)', r')*\1', replaced_str)  # ')' before `pl.col`
    print(f"After Adding * Between Parentheses and pl.col: {replaced_str}")
    
    replaced_str = re.sub(r'(\d)(pl\.col)', r'\1*\2', replaced_str)  # Number before `pl.col`
    print(f"After Adding * Between Numbers and pl.col: {replaced_str}")
    
    return replaced_str

# Example usage
input_expr = '0.25(0.1CU+0.5NI+0.6CR+0.7MO)+0.8SN'
output_expr = convert_to_polars_expr(input_expr)
print(f"Final Output: {output_expr}")


After Column Replacement: 0.25(0.1pl.col('CU')+0.5pl.col('NI')+0.6pl.col('CR')+0.7pl.col('MO'))+0.8pl.col('SN')
After Adding * Before (: 0.25*(0.1pl.col('CU')+0.5pl.col('NI')+0.6pl.col('CR')+0.7pl.col('MO'))+0.8pl.col('SN')
After Adding * Between Parentheses and pl.col: 0.25*(0.1pl.col('CU')+0.5pl.col('NI')+0.6pl.col('CR')+0.7pl.col('MO'))+0.8pl.col('SN')
After Adding * Between Numbers and pl.col: 0.25*(0.1*pl.col('CU')+0.5*pl.col('NI')+0.6*pl.col('CR')+0.7*pl.col('MO'))+0.8*pl.col('SN')
Final Output: 0.25*(0.1*pl.col('CU')+0.5*pl.col('NI')+0.6*pl.col('CR')+0.7*pl.col('MO'))+0.8*pl.col('SN')
